In [335]:
import numpy as np
import pandas as pd
import tensorflow as tf

np.random.seed(123)

In [336]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
labels = train.pop('Survived')
test_id = test.PassengerId

train.shape, test.shape

((891, 11), (418, 11))

In [337]:
train_rows = len(train)
data = pd.concat([train, test])
del train
del test

In [338]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [339]:
data.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [340]:
# function to assign sex cateogry for adults and children
def assign_sex(age, sex, value):
    if age >=18:
        if sex == value:
            return True
        return False
    return False

# function to assign title for a name
def get_pos(nam, dic):
    nam = nam.split(',')[1]
    first_nam = nam.split('.')[0].strip()
    return dic[first_nam]


# main function to apply all preprocessing
def preprocessing(X):
    
    # dropping unecessary columns
    try:
        X.drop(['PassengerId', 'Ticket'], axis=1, inplace=True)
    except:
        pass

    
    # handling null values and normalizing values
    from sklearn.preprocessing import Imputer, MinMaxScaler
    im = Imputer(missing_values='NaN', strategy='median')
    X["Age"] = im.fit_transform(X[["Age"]])

    im = Imputer(missing_values='NaN', strategy='mean')
    X["Fare"] = im.fit_transform(X[['Fare']])

    mm = MinMaxScaler()
    X['Fare'] = mm.fit_transform(X[['Fare']])
    X['Age'] = mm.fit_transform(X[['Age']])
    
    
    # Creating new column Child
    X['Child'], X['Man'], X['Woman'] = 0, 0, 0
    X['Child'] = X['Age'].apply(lambda age: 1 if age<18 else 0)
    X['Man'] = X.apply(lambda row: 1 if assign_sex(row["Age"], row["Sex"], 'male') else 0, axis=1)
    X['Woman'] = X.apply(lambda row: 1 if assign_sex(row["Age"], row["Sex"], 'female') else 0, axis=1)
    
    
    X['Cabin'] = X['Cabin'].apply(lambda x: 0 if type(x)==float else 1)  
    
    
    # getting titles inplace of names   
    dic = {             "Capt":       "Officer",
                        "Col":        "Officer",
                        "Major":      "Officer",
                        "Jonkheer":   "Royalty",
                        "Don":        "Royalty",
                        "Sir" :       "Royalty",
                        "Dr":         "Officer",
                        "Rev":        "Officer",
                        "the Countess":"Royalty",
                        "Dona":       "Royalty",
                        "Mme":        "Mrs",
                        "Mlle":       "Miss",
                        "Ms":         "Mrs",
                        "Mr" :        "Mr",
                        "Mrs" :       "Mrs",
                        "Miss" :      "Miss",
                        "Master" :    "Master",
                        "Lady" :      "Royalty"}
    
    title = X['Name'].apply(lambda nam: get_pos(nam, dic))
    df_title = pd.get_dummies(title, prefix='Title')
    X = pd.concat([df_title, X], axis=1)
    
    X['Embarked'].fillna(value='S', inplace=True)
    df_embarked = pd.get_dummies(X['Embarked'], prefix='Embarked')
    X = pd.concat([df_embarked, X], axis=1)
    
    df_class = pd.get_dummies(X['Pclass'], prefix='Pclass')
    X = pd.concat([df_class, X], axis=1)
    
    X.drop(['Name', 'Sex', 'Embarked', 'Pclass'], axis=1, inplace=True)
    X.index += 1
    
    return X

In [341]:
data = preprocessing(data)
data.shape

(1309, 20)

In [342]:
train = data[:train_rows]
test = data[train_rows:]
del data

In [343]:
def data_for_nn(data, labels):
    data = data.values
    if labels is not None:
        labels = (np.arange(2) == labels[:, None]).astype(np.float32)
        
    return data, labels

In [344]:
train, labels = data_for_nn(train, labels)
X_test, _ = data_for_nn(test, None)

In [345]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, labels, test_size=0.15, stratify=labels)
X_train.shape, X_val.shape

((757, 20), (134, 20))

In [346]:
tf.reset_default_graph()

n1, n2, n3 = 20, 10, 5

X = tf.placeholder(tf.float32, shape=[None, n1])

w1 = tf.Variable(tf.truncated_normal([n1, n2], stddev=0.1))
w2 = tf.Variable(tf.truncated_normal([n2, n3], stddev=0.1))
w3 = tf.Variable(tf.truncated_normal([n3, 2], stddev=0.1))

b1 = tf.Variable(tf.zeros([n2]))
b2 = tf.Variable(tf.zeros([n3]))
b3 = tf.Variable(tf.zeros([2]))


y1 = tf.nn.relu(tf.matmul(X, w1)+b1)
y2 = tf.nn.relu(tf.matmul(y1, w2)+b2)

Y = tf.matmul(y2, w3)+b3

In [347]:
Y_true = tf.placeholder(tf.float32, shape=[None, 2])

In [348]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=Y_true)
mean_loss = tf.reduce_mean(loss)

is_correct = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_true, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [349]:
global_step = tf.Variable(0, trainable=False)
'''starter_learning_rate = 0.003

learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 
                                           500, 0.7, staircase=True)
'''
learning_rate = 0.0001

In [350]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_step = optimizer.minimize(mean_loss, global_step=global_step)

In [351]:
sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

In [352]:
for epoch_counter in range(3000):
    train_data = {X: X_train, Y_true: y_train}
    _, train_loss = sess.run([train_step, mean_loss], feed_dict=train_data)
    
    val_data = {X: X_val, Y_true: y_val}
    val_loss, val_acc = sess.run([mean_loss, accuracy], feed_dict=val_data)
    
    print ("Epoch %d:  Train_loss=%0.3f  Val_loss=%0.3f  Val_Acc=%0.3f  eta=%0.5f global_step=%d"
          %(epoch_counter+1,
            train_loss,
            val_loss,
            val_acc,
            learning_rate,
            global_step.eval(session=sess)))

Epoch 1:  Train_loss=0.693  Val_loss=0.693  Val_Acc=0.612  eta=0.00010 global_step=1
Epoch 2:  Train_loss=0.693  Val_loss=0.693  Val_Acc=0.619  eta=0.00010 global_step=2
Epoch 3:  Train_loss=0.693  Val_loss=0.693  Val_Acc=0.619  eta=0.00010 global_step=3
Epoch 4:  Train_loss=0.693  Val_loss=0.693  Val_Acc=0.619  eta=0.00010 global_step=4
Epoch 5:  Train_loss=0.693  Val_loss=0.692  Val_Acc=0.619  eta=0.00010 global_step=5
Epoch 6:  Train_loss=0.693  Val_loss=0.692  Val_Acc=0.619  eta=0.00010 global_step=6
Epoch 7:  Train_loss=0.693  Val_loss=0.692  Val_Acc=0.619  eta=0.00010 global_step=7
Epoch 8:  Train_loss=0.692  Val_loss=0.692  Val_Acc=0.619  eta=0.00010 global_step=8
Epoch 9:  Train_loss=0.692  Val_loss=0.692  Val_Acc=0.619  eta=0.00010 global_step=9
Epoch 10:  Train_loss=0.692  Val_loss=0.692  Val_Acc=0.619  eta=0.00010 global_step=10
Epoch 11:  Train_loss=0.692  Val_loss=0.692  Val_Acc=0.619  eta=0.00010 global_step=11
Epoch 12:  Train_loss=0.692  Val_loss=0.692  Val_Acc=0.619  e

Epoch 149:  Train_loss=0.687  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=149
Epoch 150:  Train_loss=0.687  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=150
Epoch 151:  Train_loss=0.687  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=151
Epoch 152:  Train_loss=0.687  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=152
Epoch 153:  Train_loss=0.687  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=153
Epoch 154:  Train_loss=0.687  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=154
Epoch 155:  Train_loss=0.687  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=155
Epoch 156:  Train_loss=0.687  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=156
Epoch 157:  Train_loss=0.687  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=157
Epoch 158:  Train_loss=0.686  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=158
Epoch 159:  Train_loss=0.686  Val_loss=0.686  Val_Acc=0.619  eta=0.00010 global_step=159
Epoch 160:  Train_los

Epoch 290:  Train_loss=0.676  Val_loss=0.674  Val_Acc=0.619  eta=0.00010 global_step=290
Epoch 291:  Train_loss=0.675  Val_loss=0.674  Val_Acc=0.619  eta=0.00010 global_step=291
Epoch 292:  Train_loss=0.675  Val_loss=0.673  Val_Acc=0.619  eta=0.00010 global_step=292
Epoch 293:  Train_loss=0.675  Val_loss=0.673  Val_Acc=0.619  eta=0.00010 global_step=293
Epoch 294:  Train_loss=0.675  Val_loss=0.673  Val_Acc=0.619  eta=0.00010 global_step=294
Epoch 295:  Train_loss=0.675  Val_loss=0.673  Val_Acc=0.619  eta=0.00010 global_step=295
Epoch 296:  Train_loss=0.675  Val_loss=0.673  Val_Acc=0.619  eta=0.00010 global_step=296
Epoch 297:  Train_loss=0.675  Val_loss=0.673  Val_Acc=0.619  eta=0.00010 global_step=297
Epoch 298:  Train_loss=0.675  Val_loss=0.673  Val_Acc=0.619  eta=0.00010 global_step=298
Epoch 299:  Train_loss=0.675  Val_loss=0.673  Val_Acc=0.619  eta=0.00010 global_step=299
Epoch 300:  Train_loss=0.674  Val_loss=0.672  Val_Acc=0.619  eta=0.00010 global_step=300
Epoch 301:  Train_los

Epoch 438:  Train_loss=0.650  Val_loss=0.645  Val_Acc=0.619  eta=0.00010 global_step=438
Epoch 439:  Train_loss=0.649  Val_loss=0.645  Val_Acc=0.619  eta=0.00010 global_step=439
Epoch 440:  Train_loss=0.649  Val_loss=0.645  Val_Acc=0.619  eta=0.00010 global_step=440
Epoch 441:  Train_loss=0.649  Val_loss=0.644  Val_Acc=0.619  eta=0.00010 global_step=441
Epoch 442:  Train_loss=0.649  Val_loss=0.644  Val_Acc=0.619  eta=0.00010 global_step=442
Epoch 443:  Train_loss=0.648  Val_loss=0.644  Val_Acc=0.619  eta=0.00010 global_step=443
Epoch 444:  Train_loss=0.648  Val_loss=0.644  Val_Acc=0.619  eta=0.00010 global_step=444
Epoch 445:  Train_loss=0.648  Val_loss=0.643  Val_Acc=0.619  eta=0.00010 global_step=445
Epoch 446:  Train_loss=0.648  Val_loss=0.643  Val_Acc=0.619  eta=0.00010 global_step=446
Epoch 447:  Train_loss=0.648  Val_loss=0.643  Val_Acc=0.619  eta=0.00010 global_step=447
Epoch 448:  Train_loss=0.647  Val_loss=0.643  Val_Acc=0.619  eta=0.00010 global_step=448
Epoch 449:  Train_los

Epoch 585:  Train_loss=0.612  Val_loss=0.604  Val_Acc=0.619  eta=0.00010 global_step=585
Epoch 586:  Train_loss=0.611  Val_loss=0.604  Val_Acc=0.619  eta=0.00010 global_step=586
Epoch 587:  Train_loss=0.611  Val_loss=0.604  Val_Acc=0.619  eta=0.00010 global_step=587
Epoch 588:  Train_loss=0.611  Val_loss=0.604  Val_Acc=0.619  eta=0.00010 global_step=588
Epoch 589:  Train_loss=0.611  Val_loss=0.603  Val_Acc=0.619  eta=0.00010 global_step=589
Epoch 590:  Train_loss=0.610  Val_loss=0.603  Val_Acc=0.619  eta=0.00010 global_step=590
Epoch 591:  Train_loss=0.610  Val_loss=0.603  Val_Acc=0.619  eta=0.00010 global_step=591
Epoch 592:  Train_loss=0.610  Val_loss=0.602  Val_Acc=0.619  eta=0.00010 global_step=592
Epoch 593:  Train_loss=0.610  Val_loss=0.602  Val_Acc=0.619  eta=0.00010 global_step=593
Epoch 594:  Train_loss=0.609  Val_loss=0.602  Val_Acc=0.619  eta=0.00010 global_step=594
Epoch 595:  Train_loss=0.609  Val_loss=0.601  Val_Acc=0.619  eta=0.00010 global_step=595
Epoch 596:  Train_los

...

Epoch 2457:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2457
Epoch 2458:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2458
Epoch 2459:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2459
Epoch 2460:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2460
Epoch 2461:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2461
Epoch 2462:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2462
Epoch 2463:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2463
Epoch 2464:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2464
Epoch 2465:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2465
Epoch 2466:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2466
Epoch 2467:  Train_loss=0.413  Val_loss=0.398  Val_Acc=0.836  eta=0.00010 global_step=2467

Epoch 2600:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.836  eta=0.00010 global_step=2600
Epoch 2601:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.836  eta=0.00010 global_step=2601
Epoch 2602:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.836  eta=0.00010 global_step=2602
Epoch 2603:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.836  eta=0.00010 global_step=2603
Epoch 2604:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.836  eta=0.00010 global_step=2604
Epoch 2605:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.836  eta=0.00010 global_step=2605
Epoch 2606:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.836  eta=0.00010 global_step=2606
Epoch 2607:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.836  eta=0.00010 global_step=2607
Epoch 2608:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.836  eta=0.00010 global_step=2608
Epoch 2609:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.828  eta=0.00010 global_step=2609
Epoch 2610:  Train_loss=0.410  Val_loss=0.396  Val_Acc=0.828  eta=0.00010 global_step=2610

Epoch 2734:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2734
Epoch 2735:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2735
Epoch 2736:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2736
Epoch 2737:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2737
Epoch 2738:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2738
Epoch 2739:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2739
Epoch 2740:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2740
Epoch 2741:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2741
Epoch 2742:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2742
Epoch 2743:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2743
Epoch 2744:  Train_loss=0.407  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2744

Epoch 2845:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2845
Epoch 2846:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2846
Epoch 2847:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2847
Epoch 2848:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2848
Epoch 2849:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2849
Epoch 2850:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2850
Epoch 2851:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2851
Epoch 2852:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2852
Epoch 2853:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2853
Epoch 2854:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2854
Epoch 2855:  Train_loss=0.405  Val_loss=0.395  Val_Acc=0.828  eta=0.00010 global_step=2855

Epoch 2968:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2968
Epoch 2969:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2969
Epoch 2970:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2970
Epoch 2971:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2971
Epoch 2972:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2972
Epoch 2973:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2973
Epoch 2974:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2974
Epoch 2975:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2975
Epoch 2976:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2976
Epoch 2977:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2977
Epoch 2978:  Train_loss=0.403  Val_loss=0.394  Val_Acc=0.828  eta=0.00010 global_step=2978

In [353]:
predict = tf.argmax(tf.nn.softmax(Y), 1)
predictions = predict.eval(feed_dict={X: X_test})

In [354]:
submission = pd.DataFrame({'PassengerId': test_id, 'Survived':predictions})
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [355]:
submission.to_csv('my_submission.csv',index=False)